# Week 24: Capstone Trading Strategy

## Multi-Asset ML Trading System - Complete Implementation

### Overview
This notebook integrates ALL concepts from the 24-week curriculum into a production-ready trading system.

### Tickers: AAPL, MSFT, GOOGL, JPM, GS

### Features
- Multi-model ensemble (Linear, Tree, Neural Network)
- Factor-based alpha generation
- Risk-adjusted position sizing
- Options signals (implied volatility analysis)
- Full backtesting with transaction costs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

import yfinance as yf
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from enum import Enum

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')
print("✅ Libraries loaded successfully!")

✅ Libraries loaded successfully!


## 1. Configuration & Data Loading

In [2]:
# Configuration
class Config:
    TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'JPM', 'GS']
    START_DATE = '2019-01-01'
    END_DATE = datetime.now().strftime('%Y-%m-%d')
    INITIAL_CAPITAL = 100_000
    TRANSACTION_COST = 0.001  # 10 bps
    SLIPPAGE = 0.0005  # 5 bps
    MAX_POSITION_SIZE = 0.25  # Max 25% per position
    STOP_LOSS = 0.05  # 5% stop loss
    TAKE_PROFIT = 0.15  # 15% take profit
    RISK_FREE_RATE = 0.05

# Download data
print("📥 Downloading market data...")
data = yf.download(
    Config.TICKERS, 
    start=Config.START_DATE, 
    end=Config.END_DATE,
    progress=False,
    auto_adjust=True
)

prices = data['Close'].dropna()
volume = data['Volume'].dropna()
high = data['High'].dropna()
low = data['Low'].dropna()
returns = prices.pct_change().dropna()

print(f"✅ Data loaded: {len(prices)} days, {len(Config.TICKERS)} tickers")
print(f"   Date range: {prices.index[0].date()} to {prices.index[-1].date()}")

📥 Downloading market data...
✅ Data loaded: 1775 days, 5 tickers
   Date range: 2019-01-02 to 2026-01-23


## 2. Feature Engineering Pipeline

In [3]:
class FeatureEngineering:
    """Comprehensive feature engineering for trading signals."""
    
    @staticmethod
    def momentum_features(prices: pd.DataFrame, returns: pd.DataFrame) -> pd.DataFrame:
        """Calculate momentum-based features."""
        features = pd.DataFrame(index=prices.index)
        
        for ticker in prices.columns:
            # Price momentum
            features[f'{ticker}_mom_5d'] = prices[ticker].pct_change(5)
            features[f'{ticker}_mom_20d'] = prices[ticker].pct_change(20)
            features[f'{ticker}_mom_60d'] = prices[ticker].pct_change(60)
            
            # Moving average crossovers
            sma_20 = prices[ticker].rolling(20).mean()
            sma_50 = prices[ticker].rolling(50).mean()
            sma_200 = prices[ticker].rolling(200).mean()
            
            features[f'{ticker}_price_sma20'] = prices[ticker] / sma_20 - 1
            features[f'{ticker}_sma20_sma50'] = sma_20 / sma_50 - 1
            features[f'{ticker}_sma50_sma200'] = sma_50 / sma_200 - 1
            
            # Return ranking
            features[f'{ticker}_ret_rank'] = returns[ticker].rolling(20).apply(
                lambda x: (x.iloc[-1] > x).mean()
            )
        
        return features
    
    @staticmethod
    def volatility_features(prices: pd.DataFrame, returns: pd.DataFrame,
                           high: pd.DataFrame, low: pd.DataFrame) -> pd.DataFrame:
        """Calculate volatility-based features."""
        features = pd.DataFrame(index=prices.index)
        
        for ticker in prices.columns:
            # Historical volatility
            features[f'{ticker}_vol_20d'] = returns[ticker].rolling(20).std() * np.sqrt(252)
            features[f'{ticker}_vol_60d'] = returns[ticker].rolling(60).std() * np.sqrt(252)
            
            # Volatility ratio
            features[f'{ticker}_vol_ratio'] = (
                features[f'{ticker}_vol_20d'] / features[f'{ticker}_vol_60d']
            )
            
            # Parkinson volatility (high-low)
            features[f'{ticker}_parkinson'] = np.sqrt(
                (1 / (4 * np.log(2))) * 
                (np.log(high[ticker] / low[ticker]) ** 2).rolling(20).mean()
            ) * np.sqrt(252)
            
            # ATR
            tr = pd.DataFrame({
                'hl': high[ticker] - low[ticker],
                'hc': abs(high[ticker] - prices[ticker].shift(1)),
                'lc': abs(low[ticker] - prices[ticker].shift(1))
            }).max(axis=1)
            features[f'{ticker}_atr'] = tr.rolling(14).mean() / prices[ticker]
        
        return features
    
    @staticmethod
    def technical_features(prices: pd.DataFrame, volume: pd.DataFrame) -> pd.DataFrame:
        """Calculate technical indicators."""
        features = pd.DataFrame(index=prices.index)
        
        for ticker in prices.columns:
            # RSI
            delta = prices[ticker].diff()
            gain = delta.where(delta > 0, 0).rolling(14).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
            rs = gain / loss
            features[f'{ticker}_rsi'] = 100 - (100 / (1 + rs))
            
            # MACD
            ema_12 = prices[ticker].ewm(span=12).mean()
            ema_26 = prices[ticker].ewm(span=26).mean()
            macd = ema_12 - ema_26
            signal = macd.ewm(span=9).mean()
            features[f'{ticker}_macd'] = (macd - signal) / prices[ticker]
            
            # Bollinger Bands
            sma = prices[ticker].rolling(20).mean()
            std = prices[ticker].rolling(20).std()
            features[f'{ticker}_bb_position'] = (prices[ticker] - sma) / (2 * std)
            
            # Volume features
            features[f'{ticker}_vol_sma'] = volume[ticker] / volume[ticker].rolling(20).mean()
            features[f'{ticker}_obv'] = (
                (np.sign(prices[ticker].diff()) * volume[ticker]).cumsum().diff(20) /
                volume[ticker].rolling(20).sum()
            )
        
        return features
    
    @staticmethod
    def cross_sectional_features(returns: pd.DataFrame) -> pd.DataFrame:
        """Calculate cross-sectional features."""
        features = pd.DataFrame(index=returns.index)
        
        # Cross-sectional rank
        ranks = returns.rolling(20).mean().rank(axis=1, pct=True)
        for ticker in returns.columns:
            features[f'{ticker}_xs_rank'] = ranks[ticker]
        
        # Relative strength
        for ticker in returns.columns:
            features[f'{ticker}_rel_strength'] = (
                returns[ticker].rolling(20).mean() - returns.rolling(20).mean().mean(axis=1)
            )
        
        return features

# Build feature matrix
print("🔧 Building feature matrix...")

fe = FeatureEngineering()
features = pd.concat([
    fe.momentum_features(prices, returns),
    fe.volatility_features(prices, returns, high, low),
    fe.technical_features(prices, volume),
    fe.cross_sectional_features(returns)
], axis=1)

# Clean features
features = features.replace([np.inf, -np.inf], np.nan)
features = features.dropna()

print(f"✅ Features built: {features.shape[1]} features, {len(features)} observations")

🔧 Building feature matrix...
✅ Features built: 95 features, 1576 observations


## 3. Multi-Model Ensemble

In [ ]:
class Signal(Enum):
    STRONG_BUY = 2
    BUY = 1
    HOLD = 0
    SELL = -1
    STRONG_SELL = -2

@dataclass
class ModelPrediction:
    ticker: str
    signal: Signal
    confidence: float
    probability: float
    model_name: str

class EnsembleModel:
    """Multi-model ensemble for trading signals."""
    
    def __init__(self):
        self.models = {
            'ridge': Ridge(alpha=1.0),
            'logistic': LogisticRegression(C=0.1, max_iter=1000),
            'rf': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),
            'gbm': GradientBoostingClassifier(n_estimators=100, max_depth=3, random_state=42)
        }
        self.scalers = {}
        self.trained_models = {}
    
    def prepare_data(self, features: pd.DataFrame, returns: pd.DataFrame, 
                    ticker: str, lookahead: int = 5) -> Tuple[np.ndarray, np.ndarray]:
        """Prepare features and target for training."""
        # Get features for this ticker
        ticker_cols = [c for c in features.columns if ticker in c]
        X = features[ticker_cols].copy()
        
        # Create target: future returns
        future_ret = returns[ticker].shift(-lookahead)
        y = (future_ret > 0).astype(int)
        
        # Align data
        common_idx = X.index.intersection(y.dropna().index)
        X = X.loc[common_idx]
        y = y.loc[common_idx]
        
        return X, y
    
    def train(self, features: pd.DataFrame, returns: pd.DataFrame, 
             ticker: str, train_end: str) -> None:
        """Train all models for a ticker."""
        X, y = self.prepare_data(features, returns, ticker)
        
        # Split train/test
        train_mask = X.index <= train_end
        X_train = X[train_mask]
        y_train = y[train_mask]
        
        # Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        self.scalers[ticker] = scaler
        
        # Train models
        self.trained_models[ticker] = {}
        for name, model in self.models.items():
            model_copy = type(model)(**model.get_params())
            if name == 'ridge':
                model_copy.fit(X_train_scaled, y_train.astype(float))
            else:
                model_copy.fit(X_train_scaled, y_train)
            self.trained_models[ticker][name] = model_copy
    
    def predict(self, features: pd.DataFrame, ticker: str) -> Dict[str, float]:
        """Generate predictions from all models."""
        ticker_cols = [c for c in features.columns if ticker in c]
        X = features[ticker_cols].iloc[-1:]
        X_scaled = self.scalers[ticker].transform(X)
        
        predictions = {}
        for name, model in self.trained_models[ticker].items():
            if name == 'ridge':
                pred = model.predict(X_scaled)[0]
                prob = 1 / (1 + np.exp(-pred))  # Sigmoid
            else:
                prob = model.predict_proba(X_scaled)[0, 1]
            predictions[name] = prob
        
        return predictions
    
    def ensemble_signal(self, predictions: Dict[str, float]) -> Tuple[Signal, float]:
        """Combine model predictions into ensemble signal."""
        # Weighted average (give more weight to ensemble methods)
        weights = {'ridge': 0.15, 'logistic': 0.15, 'rf': 0.35, 'gbm': 0.35}
        ensemble_prob = sum(predictions[m] * w for m, w in weights.items())
        
        # Convert to signal
        if ensemble_prob > 0.65:
            signal = Signal.STRONG_BUY
        elif ensemble_prob > 0.55:
            signal = Signal.BUY
        elif ensemble_prob < 0.35:
            signal = Signal.STRONG_SELL
        elif ensemble_prob < 0.45:
            signal = Signal.SELL
        else:
            signal = Signal.HOLD
        
        confidence = abs(ensemble_prob - 0.5) * 2  # 0 to 1 scale
        
        return signal, confidence, ensemble_prob

# Train models
print("\n🤖 Training ensemble models...")
ensemble = EnsembleModel()

# Use 80% of data for training
train_end = features.index[int(len(features) * 0.8)].strftime('%Y-%m-%d')

for ticker in Config.TICKERS:
    ensemble.train(features, returns, ticker, train_end)
    print(f"   ✓ Trained models for {ticker}")

print("✅ All models trained!")

## 4. Risk Management & Position Sizing

In [ ]:
class RiskManager:
    """Risk management and position sizing."""
    
    def __init__(self, config: Config):
        self.config = config
    
    def calculate_var(self, returns: pd.Series, confidence: float = 0.95) -> float:
        """Calculate Value at Risk."""
        return -np.percentile(returns, (1 - confidence) * 100)
    
    def calculate_cvar(self, returns: pd.Series, confidence: float = 0.95) -> float:
        """Calculate Conditional VaR (Expected Shortfall)."""
        var = self.calculate_var(returns, confidence)
        return -returns[returns <= -var].mean()
    
    def kelly_fraction(self, win_prob: float, win_loss_ratio: float) -> float:
        """Calculate Kelly Criterion fraction."""
        if win_loss_ratio <= 0:
            return 0
        kelly = (win_prob * win_loss_ratio - (1 - win_prob)) / win_loss_ratio
        return max(0, min(kelly, 0.25))  # Cap at 25%
    
    def volatility_sizing(self, volatility: float, target_vol: float = 0.15) -> float:
        """Size position based on volatility targeting."""
        if volatility <= 0:
            return 0
        return min(target_vol / volatility, 2.0)  # Max 2x
    
    def position_size(self, signal: Signal, confidence: float, 
                     volatility: float, capital: float) -> float:
        """Calculate optimal position size."""
        # Base size from signal
        signal_weight = {
            Signal.STRONG_BUY: 1.0,
            Signal.BUY: 0.5,
            Signal.HOLD: 0.0,
            Signal.SELL: -0.5,
            Signal.STRONG_SELL: -1.0
        }[signal]
        
        # Adjust for confidence
        confidence_adj = 0.5 + 0.5 * confidence
        
        # Adjust for volatility
        vol_adj = self.volatility_sizing(volatility)
        
        # Final position size
        position_pct = signal_weight * confidence_adj * vol_adj * self.config.MAX_POSITION_SIZE
        position_value = position_pct * capital
        
        return position_value

risk_manager = RiskManager(Config)

## 5. Backtesting Engine

In [ ]:
@dataclass
class BacktestResults:
    total_return: float
    annual_return: float
    volatility: float
    sharpe_ratio: float
    max_drawdown: float
    win_rate: float
    profit_factor: float
    num_trades: int
    equity_curve: pd.Series
    positions: pd.DataFrame

class Backtester:
    """Walk-forward backtesting engine."""
    
    def __init__(self, config: Config):
        self.config = config
    
    def run(self, features: pd.DataFrame, returns: pd.DataFrame, 
           ensemble: EnsembleModel, risk_manager: RiskManager,
           start_idx: int = 0) -> BacktestResults:
        """Run backtest."""
        capital = self.config.INITIAL_CAPITAL
        positions = {ticker: 0 for ticker in self.config.TICKERS}
        entry_prices = {ticker: 0 for ticker in self.config.TICKERS}
        
        equity_curve = []
        position_history = []
        trades = []
        
        test_idx = features.index[start_idx:]
        
        for i, date in enumerate(test_idx[:-1]):
            # Get current prices
            current_prices = prices.loc[date]
            next_prices = prices.loc[test_idx[i + 1]]
            
            # Portfolio value
            portfolio_value = capital
            for ticker in self.config.TICKERS:
                if positions[ticker] != 0:
                    portfolio_value += positions[ticker] * current_prices[ticker]
            
            # Generate signals
            signals = {}
            for ticker in self.config.TICKERS:
                try:
                    preds = ensemble.predict(
                        features.loc[:date], ticker
                    )
                    signal, conf, prob = ensemble.ensemble_signal(preds)
                    signals[ticker] = (signal, conf, prob)
                except:
                    signals[ticker] = (Signal.HOLD, 0, 0.5)
            
            # Execute trades
            for ticker in self.config.TICKERS:
                signal, conf, prob = signals[ticker]
                vol = returns[ticker].loc[:date].tail(20).std() * np.sqrt(252)
                
                # Calculate target position
                target_value = risk_manager.position_size(
                    signal, conf, vol, portfolio_value
                )
                target_shares = int(target_value / current_prices[ticker]) if current_prices[ticker] > 0 else 0
                
                # Current position
                current_shares = positions[ticker]
                trade_shares = target_shares - current_shares
                
                if trade_shares != 0:
                    # Apply transaction costs
                    trade_value = abs(trade_shares * current_prices[ticker])
                    cost = trade_value * (self.config.TRANSACTION_COST + self.config.SLIPPAGE)
                    capital -= cost
                    
                    # Execute trade
                    capital -= trade_shares * current_prices[ticker]
                    positions[ticker] = target_shares
                    
                    if target_shares != 0:
                        entry_prices[ticker] = current_prices[ticker]
                    
                    trades.append({
                        'date': date,
                        'ticker': ticker,
                        'shares': trade_shares,
                        'price': current_prices[ticker],
                        'signal': signal.name
                    })
            
            # Record equity
            equity = capital
            for ticker in self.config.TICKERS:
                equity += positions[ticker] * next_prices[ticker]
            
            equity_curve.append({'date': date, 'equity': equity})
            position_history.append({
                'date': date,
                **{ticker: positions[ticker] for ticker in self.config.TICKERS}
            })
        
        # Calculate metrics
        equity_df = pd.DataFrame(equity_curve).set_index('date')['equity']
        equity_returns = equity_df.pct_change().dropna()
        
        total_return = equity_df.iloc[-1] / self.config.INITIAL_CAPITAL - 1
        days = len(equity_df)
        annual_return = (1 + total_return) ** (252 / days) - 1 if days > 0 else 0
        volatility = equity_returns.std() * np.sqrt(252)
        sharpe = (annual_return - self.config.RISK_FREE_RATE) / volatility if volatility > 0 else 0
        
        # Drawdown
        cum_max = equity_df.cummax()
        drawdown = (equity_df - cum_max) / cum_max
        max_drawdown = drawdown.min()
        
        # Win rate and profit factor
        if len(trades) > 0:
            trade_returns = [(t['shares'] * t['price']) for t in trades]
            wins = sum(1 for r in trade_returns if r > 0)
            win_rate = wins / len(trades)
            gross_profit = sum(r for r in trade_returns if r > 0)
            gross_loss = abs(sum(r for r in trade_returns if r < 0))
            profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')
        else:
            win_rate = 0
            profit_factor = 0
        
        return BacktestResults(
            total_return=total_return,
            annual_return=annual_return,
            volatility=volatility,
            sharpe_ratio=sharpe,
            max_drawdown=max_drawdown,
            win_rate=win_rate,
            profit_factor=profit_factor,
            num_trades=len(trades),
            equity_curve=equity_df,
            positions=pd.DataFrame(position_history).set_index('date')
        )

# Run backtest
print("\n📊 Running backtest...")
backtester = Backtester(Config)
start_idx = int(len(features) * 0.8)  # Start after training period

results = backtester.run(features, returns, ensemble, risk_manager, start_idx)

print(f"""
{'='*70}
                     BACKTEST RESULTS
{'='*70}

  Performance Metrics:
  ─────────────────────
  Total Return:      {results.total_return*100:>8.2f}%
  Annual Return:     {results.annual_return*100:>8.2f}%
  Volatility:        {results.volatility*100:>8.2f}%
  Sharpe Ratio:      {results.sharpe_ratio:>8.2f}
  Max Drawdown:      {results.max_drawdown*100:>8.2f}%
  
  Trading Statistics:
  ─────────────────────
  Total Trades:      {results.num_trades:>8}
  Win Rate:          {results.win_rate*100:>8.1f}%
  Profit Factor:     {results.profit_factor:>8.2f}
  
{'='*70}
""")

## 6. Live Trading Signals

In [ ]:
print("\n" + "="*70)
print("🎯 LIVE TRADING SIGNALS")
print("="*70)
print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Portfolio Capital: ${Config.INITIAL_CAPITAL:,.0f}")
print("\n")

signal_summary = []

for ticker in Config.TICKERS:
    # Get model predictions
    preds = ensemble.predict(features, ticker)
    signal, confidence, probability = ensemble.ensemble_signal(preds)
    
    # Current metrics
    current_price = prices[ticker].iloc[-1]
    vol_20d = returns[ticker].tail(20).std() * np.sqrt(252)
    mom_20d = prices[ticker].iloc[-1] / prices[ticker].iloc[-20] - 1
    
    # Position sizing
    position_value = risk_manager.position_size(
        signal, confidence, vol_20d, Config.INITIAL_CAPITAL
    )
    shares = int(abs(position_value) / current_price)
    
    # Stop loss and take profit
    stop_loss = current_price * (1 - Config.STOP_LOSS) if position_value > 0 else current_price * (1 + Config.STOP_LOSS)
    take_profit = current_price * (1 + Config.TAKE_PROFIT) if position_value > 0 else current_price * (1 - Config.TAKE_PROFIT)
    
    # Signal emoji
    signal_emoji = {
        Signal.STRONG_BUY: "🟢🟢",
        Signal.BUY: "🟢",
        Signal.HOLD: "⚪",
        Signal.SELL: "🔴",
        Signal.STRONG_SELL: "🔴🔴"
    }[signal]
    
    print(f"{'─'*25} {ticker} {'─'*25}")
    print(f"   Current Price:     ${current_price:.2f}")
    print(f"   20d Momentum:      {mom_20d*100:+.2f}%")
    print(f"   20d Volatility:    {vol_20d*100:.1f}%")
    print(f"   ─────────────────────────────")
    print(f"   Signal:            {signal_emoji} {signal.name}")
    print(f"   Confidence:        {confidence*100:.1f}%")
    print(f"   Probability:       {probability*100:.1f}%")
    print(f"   ─────────────────────────────")
    print(f"   Position Size:     ${abs(position_value):,.0f} ({shares} shares)")
    print(f"   Stop Loss:         ${stop_loss:.2f}")
    print(f"   Take Profit:       ${take_profit:.2f}")
    print(f"   ─────────────────────────────")
    print(f"   Model Predictions:")
    for model, prob in preds.items():
        print(f"     - {model:10s}: {prob*100:5.1f}%")
    print()
    
    signal_summary.append({
        'Ticker': ticker,
        'Price': current_price,
        'Signal': signal.name,
        'Confidence': f"{confidence*100:.0f}%",
        'Position': f"${abs(position_value):,.0f}"
    })

print("\n" + "="*70)
print("📋 SIGNAL SUMMARY")
print("="*70)
summary_df = pd.DataFrame(signal_summary)
print(summary_df.to_string(index=False))

## 7. Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Equity Curve
ax1 = axes[0, 0]
results.equity_curve.plot(ax=ax1, linewidth=2, color='steelblue')
ax1.axhline(Config.INITIAL_CAPITAL, color='red', linestyle='--', linewidth=1, alpha=0.5)
ax1.set_title('Equity Curve (Backtest)', fontweight='bold', fontsize=12)
ax1.set_ylabel('Portfolio Value ($)')
ax1.set_xlabel('')
ax1.legend(['Strategy', 'Initial Capital'], loc='upper left')
ax1.grid(True, alpha=0.3)

# 2. Drawdown
ax2 = axes[0, 1]
drawdown = (results.equity_curve - results.equity_curve.cummax()) / results.equity_curve.cummax()
drawdown.plot(ax=ax2, linewidth=1.5, color='crimson')
ax2.fill_between(drawdown.index, drawdown.values, 0, color='crimson', alpha=0.3)
ax2.set_title('Drawdown', fontweight='bold', fontsize=12)
ax2.set_ylabel('Drawdown (%)')
ax2.set_xlabel('')
ax2.grid(True, alpha=0.3)

# 3. Monthly Returns Heatmap
ax3 = axes[1, 0]
equity_returns = results.equity_curve.pct_change().dropna()
monthly_returns = equity_returns.resample('M').sum()
monthly_returns_pivot = monthly_returns.to_frame('return')
monthly_returns_pivot['year'] = monthly_returns_pivot.index.year
monthly_returns_pivot['month'] = monthly_returns_pivot.index.month
heatmap_data = monthly_returns_pivot.pivot(index='year', columns='month', values='return')
sns.heatmap(heatmap_data * 100, annot=True, fmt='.1f', cmap='RdYlGn', center=0, ax=ax3)
ax3.set_title('Monthly Returns (%)', fontweight='bold', fontsize=12)
ax3.set_xlabel('Month')
ax3.set_ylabel('Year')

# 4. Position Distribution
ax4 = axes[1, 1]
final_positions = results.positions.iloc[-1]
colors = ['green' if p > 0 else 'red' if p < 0 else 'gray' for p in final_positions]
ax4.bar(final_positions.index, final_positions.values, color=colors, alpha=0.7)
ax4.axhline(0, color='black', linewidth=0.5)
ax4.set_title('Final Positions (Shares)', fontweight='bold', fontsize=12)
ax4.set_ylabel('Shares')
ax4.tick_params(axis='x', rotation=45)
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('trading_strategy_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Visualization saved to trading_strategy_results.png")

## 8. Options Strategy Recommendations

In [ ]:
print("\n" + "="*70)
print("📈 OPTIONS STRATEGY RECOMMENDATIONS")
print("="*70 + "\n")

for ticker in Config.TICKERS:
    preds = ensemble.predict(features, ticker)
    signal, confidence, probability = ensemble.ensemble_signal(preds)
    vol = returns[ticker].tail(20).std() * np.sqrt(252)
    price = prices[ticker].iloc[-1]
    
    # Determine options strategy based on signal and volatility
    if signal in [Signal.STRONG_BUY, Signal.BUY]:
        if vol > 0.30:  # High vol
            strategy = "Bull Call Spread"
            strikes = f"Buy ${price*0.98:.0f}C / Sell ${price*1.05:.0f}C"
            rationale = "Bullish outlook, high IV makes spreads attractive"
        else:  # Low vol
            strategy = "Long Call"
            strikes = f"Buy ${price*1.02:.0f}C (ATM)"
            rationale = "Bullish outlook, low IV makes outright calls attractive"
    elif signal in [Signal.STRONG_SELL, Signal.SELL]:
        if vol > 0.30:
            strategy = "Bear Put Spread"
            strikes = f"Buy ${price*1.02:.0f}P / Sell ${price*0.95:.0f}P"
            rationale = "Bearish outlook, high IV makes spreads attractive"
        else:
            strategy = "Long Put"
            strikes = f"Buy ${price*0.98:.0f}P (ATM)"
            rationale = "Bearish outlook, low IV makes outright puts attractive"
    else:  # HOLD
        if vol > 0.30:
            strategy = "Iron Condor"
            strikes = f"${price*0.90:.0f}P/${price*0.95:.0f}P - ${price*1.05:.0f}C/${price*1.10:.0f}C"
            rationale = "Neutral outlook, collect premium from high IV"
        else:
            strategy = "Covered Call / Cash-Secured Put"
            strikes = f"Sell ${price*1.05:.0f}C or ${price*0.95:.0f}P"
            rationale = "Neutral outlook, generate income in low IV"
    
    print(f"🎯 {ticker}")
    print(f"   Current Price:  ${price:.2f}")
    print(f"   Volatility:     {vol*100:.1f}%")
    print(f"   Signal:         {signal.name}")
    print(f"   Strategy:       {strategy}")
    print(f"   Strikes:        {strikes}")
    print(f"   Rationale:      {rationale}")
    print()

print("\n⚠️ Options involve risk. These are educational recommendations only.")

## 9. Summary & Next Steps

### Key Achievements
- Built multi-model ensemble (Ridge, Logistic, RF, GBM)
- Implemented comprehensive feature engineering
- Created risk-adjusted position sizing
- Backtested with realistic transaction costs
- Generated actionable trading signals

### Future Enhancements
1. Add sentiment analysis from news
2. Integrate options flow data
3. Add deep learning models (LSTM, Transformer)
4. Implement reinforcement learning for execution
5. Add live trading API integration

---

**🎓 Congratulations on completing the Week 24 Capstone Trading Strategy!**